# Step 3: pair hubs together to make data easier

## Import dependencies

Code dependencies.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown


Import the trip data file.

In [2]:
# Load the trip data file
df = pd.read_csv(Path("data/all-trips-geo.csv").resolve())

df.sample(5)

,plate,Start Hub,End Hub,Date,Start Time,Trip Durration,Billing Zip Code,Start Hub Geolocation,End Hub Geolocation
1175,9140004,Arcadian Shopping Center,Arcadian Shopping Center,2024-09-15,2024-09-15 17:02:55,1:02:23,10562.0,"(41.1394616, -73.861803)","(41.1394616, -73.861803)"
1023,9140005,Metro North - Plaza,Metro North - Plaza,2024-09-11,2024-09-11 12:11:04,0:19:40,10604.0,"(41.157844, -73.868112)","(41.157844, -73.868112)"
561,9140010,Cronton Ave & Pleasantville Road,Arcadian Shopping Center,2024-10-12,2024-10-12 20:21:44,0:13:27,10562.0,"(41.1669377, -73.8497735)","(41.1394616, -73.861803)"
32,9140005,"Spring St, Cofield",Arcadian Shopping Center,2024-12-06,2024-12-06 13:43:53,0:07:13,89408.0,"(41.1570899, -73.8640002)","(41.1394616, -73.861803)"
853,9140015,Cronton Ave & Pleasantville Road,Metro North - Plaza,2024-09-23,2024-09-23 8:27:29,0:08:41,10562.0,"(41.1669377, -73.8497735)","(41.157844, -73.868112)"


Import the hub addresses

In [6]:
addresses_df = pd.read_csv(Path("data/hub-locations.csv").resolve())
addresses_df = addresses_df.drop(columns=["OpenStreetMap Data"]) # drop the column with OSM data... we don't need it here.

addresses_df.sample(5)

,Hub,Address,Geolocation
5,Municipal Parking Lot Lot# 7,"1-15 Leonard St, Ossining, NY 10562","(41.1618223, -73.8629757)"
8,Spring & Waller,"Spring St & Waller Ave, Ossining, NY 10562","(41.1593892, -73.8638232)"
0,Arcadian Shopping Center,"225-207 Albany Post Rd, Briarcliff Manor, NY 1...","(41.1394616, -73.861803)"
7,Ossining Public Library,"53 Croton Ave, Ossining, NY 10562","(41.1642872, -73.8604165)"
6,Nelson park,"20 Madison Ave, Ossining, NY 10562","(41.1521048, -73.8623897)"


## Aggregate data
Group together rows that have the same start and end hubs, and count the number of trips from one to the other.

In [7]:
# Group by 'Start Hub Geolocation' and 'End Hub Geolocation' and count occurrences
grouped_df = df.groupby(['Start Hub Geolocation', 'End Hub Geolocation']).size().reset_index(name='Count')
# Map geolocations to hub names using addresses_df
geo_to_hub = addresses_df.set_index('Geolocation')['Hub'].to_dict()

# Add 'Start Hub' and 'End Hub' columns with the names of the hubs
grouped_df['Start Hub'] = grouped_df['Start Hub Geolocation'].map(geo_to_hub)
grouped_df['End Hub'] = grouped_df['End Hub Geolocation'].map(geo_to_hub)

grouped_df.head()

,Start Hub Geolocation,End Hub Geolocation,Count,Start Hub,End Hub
0,"(41.1394616, -73.861803)","(41.1394616, -73.861803)",40,Arcadian Shopping Center,Arcadian Shopping Center
1,"(41.1394616, -73.861803)","(41.1521048, -73.8623897)",8,Arcadian Shopping Center,Nelson park
2,"(41.1394616, -73.861803)","(41.1570899, -73.8640002)",4,Arcadian Shopping Center,"Spring St, Cofield"
3,"(41.1394616, -73.861803)","(41.157844, -73.868112)",8,Arcadian Shopping Center,Metro North - Plaza
4,"(41.1394616, -73.861803)","(41.1593892, -73.8638232)",16,Arcadian Shopping Center,Spring & Waller


Mark whether the start and end hubs are the same.

In [36]:
# Add a new column 'One Hub' to indicate if 'Start Hub' and 'End Hub' are the same
grouped_df['Single Hub'] = grouped_df['Start Hub'] == grouped_df['End Hub']

grouped_df.sort_values(by='Count', inplace=True, ascending=False)
grouped_df[['Count', 'Start Hub', 'End Hub', 'Single Hub']].head(10) # don't show geolocations, since they're confusing

,Count,Start Hub,End Hub,Single Hub
12,159,Nelson park,Nelson park,True
70,86,Municipal Parking Lot Lot# 7,Arcadian Shopping Center,False
57,86,Wishnie Park,Wishnie Park,True
34,78,Metro North - Plaza,Metro North - Plaza,True
23,59,"Spring St, Cofield","Spring St, Cofield",True
14,58,Nelson park,Metro North - Plaza,False
100,48,C Town Markets,C Town Markets,True
76,47,Municipal Parking Lot Lot# 7,Municipal Parking Lot Lot# 7,True
111,46,Cronton Ave & Pleasantville Road,Cronton Ave & Pleasantville Road,True
7,46,Arcadian Shopping Center,Municipal Parking Lot Lot# 7,False


## Analyze each pair of hubs
Ignoring which is the start and which is the end... for now.  So we can see total trips between each pair, regardless of direction.

In [37]:
# Create a new column with sorted tuples of 'Start Hub' and 'End Hub'
grouped_df['Hub Pair Sorted'] = grouped_df.apply(lambda row: tuple(sorted([row['Start Hub'], row['End Hub']])), axis=1)

# Group by the sorted hub pairs and sum the 'Count' values
combined_df = grouped_df.groupby('Hub Pair Sorted', as_index=False).agg({'Count': 'sum'})

# Rename columns for clarity
combined_df.rename(columns={'Hub Pair Sorted': 'Hub Pair', 'Count': 'Total Count'}, inplace=True)

# Sort by Total Count
combined_df.sort_values(by='Total Count', ascending=False, inplace=True)

combined_df.head()

,Hub Pair,Total Count
49,"(Nelson park, Nelson park)",159
5,"(Arcadian Shopping Center, Municipal Parking L...",132
38,"(Metro North - Plaza, Nelson park)",92
62,"(Wishnie Park, Wishnie Park)",86
36,"(Metro North - Plaza, Metro North - Plaza)",78


Add back in the geolocations of each hub in the pair.

In [26]:
# Create a dictionary mapping Hub names to their Geolocation
hub_to_geo = addresses_df.set_index('Hub')['Geolocation'].to_dict()

# Add 'Hub 1 Geolocation' and 'Hub 2 Geolocation' columns
combined_df['Hub 1 Geolocation'] = combined_df['Hub Pair'].apply(lambda pair: hub_to_geo[pair[0]])
combined_df['Hub 2 Geolocation'] = combined_df['Hub Pair'].apply(lambda pair: hub_to_geo[pair[1]])

combined_df.head()

,Hub Pair,Total Count,Hub 1 Geolocation,Hub 2 Geolocation
49,"(Nelson park, Nelson park)",159,"(41.1521048, -73.8623897)","(41.1521048, -73.8623897)"
5,"(Arcadian Shopping Center, Municipal Parking L...",132,"(41.1394616, -73.861803)","(41.1618223, -73.8629757)"
38,"(Metro North - Plaza, Nelson park)",92,"(41.157844, -73.868112)","(41.1521048, -73.8623897)"
62,"(Wishnie Park, Wishnie Park)",86,"(41.1607067, -73.8477673)","(41.1607067, -73.8477673)"
36,"(Metro North - Plaza, Metro North - Plaza)",78,"(41.157844, -73.868112)","(41.157844, -73.868112)"


Mark how many trips had the first hub as the start and the second as the end.

In [ ]:
# Create a dictionary mapping (Start Hub, End Hub) to Count from grouped_df
start_count_map = grouped_df.set_index(['Start Hub', 'End Hub'])['Count'].to_dict()

# Add 'Hub 1 Start Count' to combined_df
combined_df['Hub 1 Start Count'] = combined_df['Hub Pair'].apply(lambda pair: start_count_map.get((pair[0], pair[1]), 0))

combined_df.head()

,Hub Pair,Total Count,Hub 1 Geolocation,Hub 2 Geolocation,Hub 1 Start Count
49,"(Nelson park, Nelson park)",159,"(41.1521048, -73.8623897)","(41.1521048, -73.8623897)",159
5,"(Arcadian Shopping Center, Municipal Parking L...",132,"(41.1394616, -73.861803)","(41.1618223, -73.8629757)",46
38,"(Metro North - Plaza, Nelson park)",92,"(41.157844, -73.868112)","(41.1521048, -73.8623897)",34
62,"(Wishnie Park, Wishnie Park)",86,"(41.1607067, -73.8477673)","(41.1607067, -73.8477673)",86
36,"(Metro North - Plaza, Metro North - Plaza)",78,"(41.157844, -73.868112)","(41.157844, -73.868112)",78


Mark how many trips had the second hub as the start and the first as the end.

In [28]:
# Add 'Hub 2 Start Count' to combined_df
combined_df['Hub 2 Start Count'] = combined_df['Hub Pair'].apply(lambda pair: start_count_map.get((pair[1], pair[0]), 0))

combined_df.head()

,Hub Pair,Total Count,Hub 1 Geolocation,Hub 2 Geolocation,Hub 1 Start Count,Hub 2 Start Count
49,"(Nelson park, Nelson park)",159,"(41.1521048, -73.8623897)","(41.1521048, -73.8623897)",159,159
5,"(Arcadian Shopping Center, Municipal Parking L...",132,"(41.1394616, -73.861803)","(41.1618223, -73.8629757)",46,86
38,"(Metro North - Plaza, Nelson park)",92,"(41.157844, -73.868112)","(41.1521048, -73.8623897)",34,58
62,"(Wishnie Park, Wishnie Park)",86,"(41.1607067, -73.8477673)","(41.1607067, -73.8477673)",86,86
36,"(Metro North - Plaza, Metro North - Plaza)",78,"(41.157844, -73.868112)","(41.157844, -73.868112)",78,78


Add back in whether the start and end hubs are the same, since we've lost that.

In [38]:
# Add a 'Single Hub' field to indicate if both hubs in 'Hub Pair' are the same
combined_df['Single Hub'] = combined_df['Hub Pair'].apply(lambda pair: pair[0] == pair[1])

combined_df.head()

,Hub Pair,Total Count,Single Hub
49,"(Nelson park, Nelson park)",159,True
5,"(Arcadian Shopping Center, Municipal Parking L...",132,False
38,"(Metro North - Plaza, Nelson park)",92,False
62,"(Wishnie Park, Wishnie Park)",86,True
36,"(Metro North - Plaza, Metro North - Plaza)",78,True


## Save
Save the hub pair stats to a CSV file for later use.

In [31]:
# Save to file
file_path = './data/hub-pair-stats.csv'
file_path = Path(file_path).resolve() # make platform-agnostic
combined_df.to_csv(file_path, index=False)